In [4]:
import numpy as np
import os
import random
from os.path import join
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation
from keras.utils import to_categorical
from image_generators import train_generator_single_images, valid_generator_single_images
from keras_utils import set_keras_session

In [5]:
set_keras_session()
dataset = 'UCF11'
nb_classes = int(dataset[3:])

In [12]:
width = 320
height = 240
channels = 3
frame_number = 50

image = (height, width, channels)

dataset_dir = join('datasets', dataset, ('separate_frames_{}_h_{}_w_{}').format(frame_number, height, width))

batch_size = 32
train_generator = train_generator_single_images(dataset_dir, batch_size)
valid_generator = valid_generator_single_images(dataset_dir)

In [25]:
def build_model():
    model = Sequential()
    
    model.add(Conv2D(64, (3,3), activation='relu', input_shape=(height, width, channels)))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    
    return model

In [26]:
from keras import metrics
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

model = build_model()
print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 238, 318, 64)      1792      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 236, 316, 64)      36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 118, 158, 64)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 118, 158, 64)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 116, 156, 128)     73856     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 114, 154, 128)     147584    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 57, 77, 128)       0         
__________

In [28]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=int(1600 * 0.8) * frame_number / batch_size, 
                    validation_data=valid_generator,
                    validation_steps=320,
                    epochs=3)

Epoch 1/3
 702/2000 [=========>....................] - ETA: 6:44 - loss: 2.3983 - acc: 0.0893 - top_3: 0.2716

KeyboardInterrupt: 